In [1]:
import tensorflow as tf
import numpy as np

def rand01(digit):
	# Add some random noise to bits, but keep always between 0 and 1
	s = abs(np.random.normal(0.0, 0.05))
	if digit == 0:
		noise = digit + s
	else:
		noise = digit - s
	return noise

### Training Examples
### All combinations of XOR

X = [[0, 0],[0, 1],[1, 0],[1, 1]]
Y = [[1], [0], [0], [1]]

# Add some random noise to our inputs. Useful if we use the tanh activiation function

add_noise = np.vectorize(rand01)  
X = add_noise(X)
Y = add_noise(Y)

# Neural Network Parameters

N_STEPS = 200000
N_EPOCH = 5000
N_TRAINING = len(X)

N_INPUT_NODES = 2
N_HIDDEN_NODES = 5
N_OUTPUT_NODES  = 1
ACTIVATION = 'tanh' # sigmoid or tanh
COST = 'ACE' # MSE or ACE
LEARNING_RATE = 0.05

if __name__ == '__main__':

	##############################################################################
	### Create placeholders for variables and define Neural Network structure  ###
	### Feed forward 3 layer, Neural Network.                                  ###
	


	x_ = tf.placeholder(tf.float32, shape=[N_TRAINING, N_INPUT_NODES], name="x-input")
	y_ = tf.placeholder(tf.float32, shape=[N_TRAINING, N_OUTPUT_NODES], name="y-input")

	theta1 = tf.Variable(tf.random_uniform([N_INPUT_NODES,N_HIDDEN_NODES], -1, 1), name="theta1")
	theta2 = tf.Variable(tf.random_uniform([N_HIDDEN_NODES,N_OUTPUT_NODES], -1, 1), name="theta2")

	bias1 = tf.Variable(tf.zeros([N_HIDDEN_NODES]), name="bias1")
	bias2 = tf.Variable(tf.zeros([N_OUTPUT_NODES]), name="bias2")


	if ACTIVATION == 'sigmoid':

		### Use a sigmoidal activation function ###

		layer1 = tf.sigmoid(tf.matmul(x_, theta1) + bias1)
		output = tf.sigmoid(tf.matmul(layer1, theta2) + bias2)

	else:
		### Use tanh activation function ###

		layer1 = tf.tanh(tf.matmul(x_, theta1) + bias1)
		output = tf.tanh(tf.matmul(layer1, theta2) + bias2)
	
		output = tf.add(output, 1)
		output = tf.multiply(output, 0.5)

	
	if COST == "MSE":

		# Mean Squared Estimate - the simplist cost function (MSE)

		cost = tf.reduce_mean(tf.square(Y - output)) 
		train_step = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(cost)
	
	else:
		# Average Cross Entropy - better behaviour and learning rate

		
		cost = - tf.reduce_mean( (y_ * tf.log(output)) + (1 - y_) * tf.log(1.0 - output)  )
		train_step = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(cost)



	init = tf.initialize_all_variables()
	sess = tf.Session()
	sess.run(init)


	for i in range(N_STEPS):
		sess.run(train_step, feed_dict={x_: X, y_: Y})
		if i % N_EPOCH == 0:
			print('Batch ', i)
			print('Inference ', sess.run(output, feed_dict={x_: X, y_: Y}))
			print('Cost ', sess.run(cost, feed_dict={x_: X, y_: Y}))
			#print('op: ', sess.run(output))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Batch  0
Inference  [[ 0.50217396]
 [ 0.27716976]
 [ 0.57263756]
 [ 0.32443747]]
Cost  0.751878
Batch  5000
Inference  [[ 0.93240523]
 [ 0.03359601]
 [ 0.02508074]
 [ 0.99011695]]
Cost  0.140809
Batch  10000
Inference  [[ 0.9324407 ]
 [ 0.03334412]
 [ 0.02482271]
 [ 0.99052489]]
Cost  0.140806
Batch  15000
Inference  [[ 0.93244004]
 [ 0.03334233]
 [ 0.02482089]
 [ 0.99052775]]
Cost  0.140806
Batch  20000
Inference  [[ 0.93244028]
 [ 0.03334147]
 [ 0.02482006]
 [ 0.9905287 ]]
Cost  0.140806
Batch  25000
Inference  [[ 0.9324404 ]
 [ 0.03334141]
 [ 0.02481991]
 [ 0.99052882]]
Cost  0.140806
Batch  30000
Inference  [[ 0.9324404 ]
 [ 0.03334141]
 [ 0.02481991]
 [ 0.99052882]]
Cost  0.140806
Batch  35000
Inference  [[ 0.9324404 ]
 [ 0.03334141]
 [ 0.02481991]
 [ 0.99052882]]
Cost  0.140806
Batch  40000
Inference  [[ 0.9324404 ]
 [ 0.03334141]
 [ 0.02481991]
 [ 0.99052882]]
Cost  0.140806
Batch  45000
Inference  [[ 0.93

In [2]:
#!/usr/bin/env PYTHONIOENCODING="utf-8" python
"""
A simple neural network learning the XNOR function
"""
import tensorflow as tf
sess = tf.InteractiveSession()

# Desired input output mapping of XNOR function:
x_ = [[0, 0], [0, 1], [1, 0], [1, 1]] # input
#labels=[0,      1,      1,      0]   # output =>
expect=[[0,1],  [1,0],  [1,0], [0,1]] # ONE HOT REPRESENTATION! 'class' [1,0]==0 [0,1]==1

# x = tf.Variable(x_)
x = tf.placeholder("float", [None,2]) #  can we feed directly?
y_ = tf.placeholder("float", [None, 2]) # two output classes

number_hidden_nodes = 20 # 20 outputs to create some room for negatives and positives

W = tf.Variable(tf.random_uniform([2, number_hidden_nodes], -.01, .01))
b = tf.Variable(tf.random_uniform([number_hidden_nodes], -.01, .01))
hidden  = tf.nn.relu(tf.matmul(x,W) + b) # first layer.

 # the XNOR function is the first nontrivial function, for which a two layer network is needed.
W2 = tf.Variable(tf.random_uniform([number_hidden_nodes,2], -.1, .1))
b2 = tf.Variable(tf.zeros([2]))
hidden2 = tf.matmul(hidden, W2)#+b2

y = tf.nn.softmax(hidden2)


# Define loss and optimizer
cross_entropy = -tf.reduce_sum(y_*tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.2).minimize(cross_entropy)

# Train
tf.initialize_all_variables().run()
for step in range(1000):
    feed_dict={x: x_, y_:expect } # feed the net with our inputs and desired outputs.
    e,a=sess.run([cross_entropy,train_step],feed_dict)
    if e<1:break # early stopping yay
    print ("step %d : entropy %s" % (step,e)) # error/loss should decrease over time


# Test trained model
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1)) # argmax along dim-1
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float")) # [True, False, True, True] -> [1,0,1,1] -> 0.75.

print ("accuracy %s"%(accuracy.eval({x: x_, y_: expect})))

learned_output=tf.argmax(y,1)
print (learned_output.eval({x: x_}))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
step 0 : entropy 2.77286
step 1 : entropy 2.77203
step 2 : entropy 2.77172
step 3 : entropy 2.7703
step 4 : entropy 2.7686
step 5 : entropy 2.76733
step 6 : entropy 2.76646
step 7 : entropy 2.76363
step 8 : entropy 2.76238
step 9 : entropy 2.76043
step 10 : entropy 2.75685
step 11 : entropy 2.75445
step 12 : entropy 2.75133
step 13 : entropy 2.74592
step 14 : entropy 2.74182
step 15 : entropy 2.73712
step 16 : entropy 2.72754
step 17 : entropy 2.72243
step 18 : entropy 2.71174
step 19 : entropy 2.70029
step 20 : entropy 2.68679
step 21 : entropy 2.67732
step 22 : entropy 2.64869
step 23 : entropy 2.64092
step 24 : entropy 2.60551
step 25 : entropy 2.58932
step 26 : entropy 2.53868
step 27 : entropy 2.53875
step 28 : entropy 2.4447
step 29 : entropy 2.45658
step 30 : entropy 2.35595
step 31 : entropy 2.35226
step 32 : entropy 2.25456
step 33 : entropy 2.18895
step 34 : entropy 2.13329
step 35 : entropy 2.02697
ste